In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/expenditure-by-salisbury-nhs-foundation-trust/expenditure.csv


In [2]:
df = pd.read_csv('../input/expenditure-by-salisbury-nhs-foundation-trust/expenditure.csv')
df.head()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7,9,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Department Family,Entity,Date,Expense Type,Expense area,Supplier,Transaction Number,Amount,VAT registration Number,Amount,VAT Registration Number,Expense,VAT registration number
0,Department of Health,Salisbury NHS Foundation Trust,2010-04-16,RATES,Facilities Directorate,WILTSHIRE COUNCIL,Apr-10,NaN,NaN,96071.17,NaN,NaN,NaN
1,Department of Health,Salisbury NHS Foundation Trust,2010-04-16,DRUGS,Clinical Support and Family Services,ROCHE PRODUCTS LTD,1321217701,NaN,NaN,25028.5,NaN,NaN,NaN
2,Department of Health,Salisbury NHS Foundation Trust,2010-04-16,DRUGS,Clinical Support and Family Services,ROCHE PRODUCTS LTD,1321217701,NaN,NaN,4379.99,NaN,NaN,NaN
3,Department of Health,Salisbury NHS Foundation Trust,2010-04-16,HEALTHCARE - NHS TRUSTS,Clinical Support and Family Services,ROYAL UNITED HOSPITAL BATH NHS TRUST,10291263,NaN,NaN,71642.5,NaN,NaN,NaN
4,Department of Health,Salisbury NHS Foundation Trust,2010-04-16,HEALTHCARE - NHS TRUSTS,Medicine,SOUTHAMPTON UNIV HOSPITAL NHS TRUST,7166068,NaN,NaN,26815.21,NaN,NaN,NaN


Let's use information from the Kaggle data explorer:

The explorer tells us that the first amount column has 56.2k valid values and 749k missing. The second amount column has 735k valid values and 56.2k missing. Therefore, the first amount column contains all the missing values of the second amount column. The expense value contains the difference.

In [3]:
df.isna().sum()

Department Family               0
Entity                          0
Date                            0
Expense Type                 9707
Expense area                   21
Supplier                        0
Transaction Number              0
 Amount                    748572
VAT registration Number    804799
Amount                      56227
VAT Registration Number    804799
Expense                    795092
VAT registration number    804799
dtype: int64

In [4]:
# Drop all VAT registration numbers since they are all missing

df.drop(["VAT registration Number", "VAT Registration Number", "VAT registration number"], axis=1, inplace=True)

In [5]:
print ("Adding the number of NaN values in the two Amount columns gives:", 748572 + 56227,". The total number of rows is {}".format(len(df)))
print ("The number of NaN values in the Expense column is:", 804799 - 795092, ".")

Adding the number of NaN values in the two Amount columns gives: 804799 . The total number of rows is 804799
The number of NaN values in the Expense column is: 9707 .


Therefore, we know that the two Amount columns should be merged and the Expense and Expense Type column should be merged.

In [6]:
# Rename columns for clarity 

df.columns = ["Department Family", "Entity", "Date", "Expense Type 1", "Expense Area", "Supplier", "Transaction Number", "Amount 1", "Amount 2", "Expense Type 2"]

In [7]:
df.head()

,Department Family,Entity,Date,Expense Type 1,Expense Area,Supplier,Transaction Number,Amount 1,Amount 2,Expense Type 2
0,Department of Health,Salisbury NHS Foundation Trust,2010-04-16,RATES,Facilities Directorate,WILTSHIRE COUNCIL,Apr-10,NaN,96071.17,NaN
1,Department of Health,Salisbury NHS Foundation Trust,2010-04-16,DRUGS,Clinical Support and Family Services,ROCHE PRODUCTS LTD,1321217701,NaN,25028.5,NaN
2,Department of Health,Salisbury NHS Foundation Trust,2010-04-16,DRUGS,Clinical Support and Family Services,ROCHE PRODUCTS LTD,1321217701,NaN,4379.99,NaN
3,Department of Health,Salisbury NHS Foundation Trust,2010-04-16,HEALTHCARE - NHS TRUSTS,Clinical Support and Family Services,ROYAL UNITED HOSPITAL BATH NHS TRUST,10291263,NaN,71642.5,NaN
4,Department of Health,Salisbury NHS Foundation Trust,2010-04-16,HEALTHCARE - NHS TRUSTS,Medicine,SOUTHAMPTON UNIV HOSPITAL NHS TRUST,7166068,NaN,26815.21,NaN


In [8]:
df["Amount 1"].fillna(df["Amount 2"], inplace=True)
df["Expense Type 1"].fillna(df["Expense Type 2"], inplace=True)

# Check all Nan filled

print (df["Amount 1"].isna().sum())
print (df["Expense Type 1"].isna().sum())

0
0


In [9]:
# Drop unnecessary columns

df.drop(["Amount 2", "Expense Type 2"], axis=1, inplace=True)

In [10]:
df.head()

,Department Family,Entity,Date,Expense Type 1,Expense Area,Supplier,Transaction Number,Amount 1
0,Department of Health,Salisbury NHS Foundation Trust,2010-04-16,RATES,Facilities Directorate,WILTSHIRE COUNCIL,Apr-10,96071.17
1,Department of Health,Salisbury NHS Foundation Trust,2010-04-16,DRUGS,Clinical Support and Family Services,ROCHE PRODUCTS LTD,1321217701,25028.5
2,Department of Health,Salisbury NHS Foundation Trust,2010-04-16,DRUGS,Clinical Support and Family Services,ROCHE PRODUCTS LTD,1321217701,4379.99
3,Department of Health,Salisbury NHS Foundation Trust,2010-04-16,HEALTHCARE - NHS TRUSTS,Clinical Support and Family Services,ROYAL UNITED HOSPITAL BATH NHS TRUST,10291263,71642.5
4,Department of Health,Salisbury NHS Foundation Trust,2010-04-16,HEALTHCARE - NHS TRUSTS,Medicine,SOUTHAMPTON UNIV HOSPITAL NHS TRUST,7166068,26815.21


In [11]:
df.rename(columns={"Expense Type 1": "Expense Type", "Amount 1": "Expenditure"}, inplace=True)
df.head()

,Department Family,Entity,Date,Expense Type,Expense Area,Supplier,Transaction Number,Expenditure
0,Department of Health,Salisbury NHS Foundation Trust,2010-04-16,RATES,Facilities Directorate,WILTSHIRE COUNCIL,Apr-10,96071.17
1,Department of Health,Salisbury NHS Foundation Trust,2010-04-16,DRUGS,Clinical Support and Family Services,ROCHE PRODUCTS LTD,1321217701,25028.5
2,Department of Health,Salisbury NHS Foundation Trust,2010-04-16,DRUGS,Clinical Support and Family Services,ROCHE PRODUCTS LTD,1321217701,4379.99
3,Department of Health,Salisbury NHS Foundation Trust,2010-04-16,HEALTHCARE - NHS TRUSTS,Clinical Support and Family Services,ROYAL UNITED HOSPITAL BATH NHS TRUST,10291263,71642.5
4,Department of Health,Salisbury NHS Foundation Trust,2010-04-16,HEALTHCARE - NHS TRUSTS,Medicine,SOUTHAMPTON UNIV HOSPITAL NHS TRUST,7166068,26815.21


In [12]:
df.to_csv('expenditure_v2.csv',index=False)